In [74]:
import numpy as np
import os
import sys
from PIL import Image
from matplotlib.pyplot import figure
import pandas
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
import torch
from torch import nn
import torch.optim as optim 
from torch.utils.data import TensorDataset, DataLoader

class MLP1(nn.Module):
  '''
    Multilayer Perceptron.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(256, 128),
      nn.ReLU(),
      nn.Linear(128, 128),
      nn.ReLU(),
      nn.Linear(128, 256),
    )


  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)

class MLP2(nn.Module):
  '''
    Multilayer Perceptron.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(1024, 256),
      nn.ReLU(),
      nn.Linear(256, 128),
      nn.ReLU(),
      nn.Linear(128,128),
      nn.ReLU(),
      nn.Linear(128,256),
    )


  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)

def predict(inputVector, weightMatrix):
        '''
        Predict the output of the input vector.
        '''
        
        predictionVector = []
        for i in range(len(weightMatrix)):
            row_sum = sum(
                (inputVector[j] * weightMatrix[j][i])
                for j in range(len(weightMatrix[0]) - 1)
            )

            predictionVector.append(self.activation_fn(row_sum))
           
        return predictionVector

def add_noise(outputVector,noise_percent,stdev):
    '''
    Add noise to the output vector.
    '''
    mean = 0
    noise = np.random.normal(mean,stdev,outputVector.shape) * noise_percent
    return outputVector + noise

def Noisy_Testing(stdev, testRounds, inputImageVectors):
    '''
    Test the DNN with noise.
    '''
    tableObject = {}
    plotObject = {'fh': [], 'ffa': []}

    for i in range(len(stdev)):
        tableObject['std_'+ str(stdev[i]) + '_fh'] = []
        tableObject['std_'+ str(stdev[i]) + '_ffa'] = []

    for j in range(len(stdev)):   
        for k in range(testRounds) :
            corruptedVector = add_noise(inputImageVectors[k],0.1,stdev[j]) 
            testPrediction = model(torch.from_numpy(corruptedVector.astype('float32'))).detach().numpy()
            for l in range(256):
                if output[l] > 0:
                    testPrediction[l] = 1
                else:
                    testPrediction[l] = 0
            fh,ffa = calculate_performance_metrics(inputImageVectors[k],testPrediction,False)
            print("FH:",fh)
            print("FFA:",ffa)

def Create_Image_Set(filename, ASL):
    ImageVectors = []
    if filename!='ASL32x':
        for i in range(10):
            path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),filename, str(i) +'.png')
            im = Image.open(path, 'r')
            gray = im.convert('L')
            bw = gray.point(lambda x: 0 if x<135 else 1, '1')
            ImageVectors.append(np.array(list(bw.getdata())))
    
    if filename!='ASL32x':
        for i in range(26):
            x = i + 65
            path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),filename, chr(i + 65) +'.png')
            im = Image.open(path, 'r')
            gray = im.convert('L')
            bw = gray.point(lambda x: 0 if x<135 else 1, '1')
            ImageVectors.append(np.array(list(bw.getdata())))
    elif filename=='ASL32x':
        for i in range(24):
            x = i + 65
            if i!=9:
                path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),filename, chr(i + 65) +'.png')
            im = Image.open(path, 'r')
            gray = im.convert('L')
            bw = gray.point(lambda x: 0 if x<135 else 1, '1')
            ImageVectors.append(np.array(list(bw.getdata())))
    elif ASL==True:
        for i in range(34):
            x = i + 65
            if i!=19 and i>10:
                path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),filename, chr(i + 65) +'.png')
            im = Image.open(path, 'r')
            gray = im.convert('L')
            bw = gray.point(lambda x: 0 if x<135 else 1, '1')
            plt.imshow(im)
            ImageVectors.append(np.array(list(bw.getdata())))

    return np.array(ImageVectors)

set1 = Create_Image_Set('characters1',False)
set2 = Create_Image_Set('characters2',False)
set3 = Create_Image_Set('ASL32x',False)
set1mod = Create_Image_Set('characters1',True)

imageTensor = torch.Tensor(set1)
_dataSet = TensorDataset(imageTensor, imageTensor)
_dataLoader = DataLoader(_dataSet)

model = MLP1()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
    losses = []

    for batch_idx, (data, targets) in enumerate(_dataLoader):

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
    
def calculate_performance_metrics(inputVector, outputVector, ASL):
    totalBlackPixelCount = sum(x == 0 for x in inputVector)
    totalWhitePixelCount = sum(x == 1 for x in inputVector)
    wrongBlackPixelCount = 0
    rightBlackPixelCount = 0
    
    if ASL==True:
        inputSize=1024
    else:
        inputSize=256
    for i in range(inputSize):
        if outputVector[i] < 0.0001:
            if  abs(outputVector[i] - inputVector[i]) < 0.0001:
                rightBlackPixelCount += 1
            else:
                wrongBlackPixelCount += 1
    fh = rightBlackPixelCount/totalBlackPixelCount
    ffa = wrongBlackPixelCount/totalWhitePixelCount
    return fh, ffa 


model.eval()
print("\n------PART 1-------\n")
print("\nChecking accuracy on Training Set\n")
for i in range(36):
    output = model(torch.from_numpy(set1[i].astype('float32'))).detach().numpy()
    for j in range(256):
        if output[j] > 0:
            output[j] = 1
        else:
            output[j] = 0
    fh, ffa = calculate_performance_metrics(set1[i], output, False)
    print("FH:", fh, "FFA:", ffa)

print("\nChecking accuracy on Test Set\n")
for i in range(36):
    output = model(torch.from_numpy(set2[i].astype('float32'))).detach().numpy()
    for j in range(256):
        if output[j] > 0:
            output[j] = 1
        else:
            output[j] = 0
    fh, ffa = calculate_performance_metrics(set2[i], output, False)
    print("FH:", fh, "FFA:", ffa)
print("\nNOISY\n")
print(part2)
stdev = [0,0.001, 0.002, 0.003, 0.005, 0.01, 0.02, 0.03, 0.05,0.1]
Noisy_Testing(stdev, 10, set1)
print("\n-------PART 2--------\n")
imageTensor = torch.Tensor(set3)
_dataSet = TensorDataset(imageTensor, imageTensor)
_dataLoader = DataLoader(_dataSet)

model = MLP2()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for i in range(24):
    output = model(torch.from_numpy(set3[i].astype('float32'))).detach().numpy()
    for j in range(256):
        if output[j] > 0:
            output[j] = 1
        else:
            output[j] = 0
    fh, ffa = calculate_performance_metrics(set3[i], output, True)
    print("FH:", fh, "FFA:", ffa)


------PART 1-------


Checking accuracy on Training Set

FH: 0.9819819819819819 FFA: 0.1310344827586207
FH: 0.9770114942528736 FFA: 0.10650887573964497
FH: 0.9106145251396648 FFA: 0.0
FH: 0.8713450292397661 FFA: 0.011764705882352941
FH: 0.8164556962025317 FFA: 0.01020408163265306
FH: 0.875 FFA: 0.011363636363636364
FH: 0.8959537572254336 FFA: 0.012048192771084338
FH: 0.9343065693430657 FFA: 0.04201680672268908
FH: 0.865979381443299 FFA: 0.08064516129032258
FH: 0.8235294117647058 FFA: 0.0
FH: 0.9865771812080537 FFA: 0.056074766355140186
FH: 0.873015873015873 FFA: 0.029850746268656716
FH: 0.9235668789808917 FFA: 0.13131313131313133
FH: 0.9157894736842105 FFA: 0.0
FH: 0.8623853211009175 FFA: 0.0
FH: 0.9109947643979057 FFA: 0.0
FH: 0.8306878306878307 FFA: 0.0
FH: 0.9619565217391305 FFA: 0.027777777777777776
FH: 0.9818181818181818 FFA: 0.03424657534246575
FH: 0.953125 FFA: 0.078125
FH: 0.9608938547486033 FFA: 0.025974025974025976
FH: 0.99375 FFA: 0.010416666666666666
FH: 0.9308755760368663

IndexError: index 256 is out of bounds for axis 0 with size 256